### https://www.youtube.com/watch?v=9vzd289Eedk&ab_channel=QuantInstiQuantitativeLearning

In [2]:
import q_tools.zerodha_authentication as zerodha_authentication
import q_credentials.zerodha_cred as zerodha_cred

In [3]:
# from kiteconnect import KiteConnect
import pandas as pd
import requests
import boto3
import io
import datetime

In [3]:
kite = zerodha_authentication.zerodha_authentication(
    api_key=zerodha_cred.api_key,
    api_secret=zerodha_cred.api_secret,
    userid=zerodha_cred.userid,
    password=zerodha_cred.password,
    pin=zerodha_cred.pin
    )

In [4]:
df_instruments=pd.DataFrame(kite.instruments())

In [11]:
s3 = boto3.client('s3',endpoint_url="http://minio-image:9000",aws_access_key_id="minio-image",aws_secret_access_key="minio-image-pass")
Bucket="airflow-files"
Key="interested_tickers_india.xlsx"
read_file = s3.get_object(Bucket=Bucket, Key=Key)
df_tickers = pd.read_excel(io.BytesIO(read_file['Body'].read()),sheet_name='nse_top_1000')

df_top=pd.merge(df_instruments,df_tickers,left_on='tradingsymbol',right_on='Symbol',how='inner')
df_top=df_top.sort_values(by=['tradingsymbol','exchange']).drop_duplicates(subset=['tradingsymbol'],keep='last')

In [13]:
df_top

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange,Sr. No.,Symbol,Company Name,Market Capitalisation \n(In lakhs)
1,3014145,11774,3IINFOTECH,3I INFOTECH.,0.0,,0.0,0.05,1,EQ,NSE,NSE,903,3IINFOTECH,3i Infotech Limited,2.101638e+04
3,121345,474,3MINDIA,3M INDIA,0.0,,0.0,0.05,1,EQ,NSE,NSE,97,3MINDIA,3M India Limited,2.120982e+06
4,138438660,540776,5PAISA,,0.0,,0.0,0.05,1,EQ,BSE,BSE,824,5PAISA,5Paisa Capital Limited,2.882841e+04
6,3038209,11868,63MOONS,63 MOONS TECHNOLOGIES,0.0,,0.0,0.05,1,EQ,NSE,NSE,901,63MOONS,63 moons technologies limited,2.121917e+04
8,1147137,4481,AARTIDRUGS,AARTI DRUGS.,0.0,,0.0,0.05,1,EQ,NSE,NSE,484,AARTIDRUGS,Aarti Drugs Limited,1.170359e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1838,1922049,7508,ZENTEC,ZEN TECHNOLOGIES,0.0,,0.0,0.05,1,EQ,NSE,NSE,939,ZENTEC,Zen Technologies Limited,1.755391e+04
1840,978177,3821,ZODIACLOTH,ZODIAC CLOTHING CO..,0.0,,0.0,0.05,1,EQ,NSE,NSE,875,ZODIACLOTH,Zodiac Clothing Company Limited,2.325316e+04
1903,2916865,11394,ZOTA,ZOTA HEALTH CARE,0.0,,0.0,0.05,1,EQ,NSE,NSE,795,ZOTA,Zota Health Care LImited,3.198983e+04
1842,7436801,29050,ZUARI,ZUARI AGRO CHEMICALS,0.0,,0.0,0.05,1,EQ,NSE,NSE,849,ZUARI,Zuari Agro Chemicals Limited,2.597082e+04


In [25]:
columns_dict={'instrument_token':'identifier','tradingsymbol':'ticker'}
df_top=df_top[['instrument_token','tradingsymbol','name','exchange']].rename(columns=columns_dict)

In [17]:
now = datetime.datetime.utcnow()

In [26]:
data_vendor_id=13

In [27]:
df_top['instrument']='Equity'
df_top['data_vendor_id']=data_vendor_id
df_top['created_date']=now
df_top['last_updated_date']=now

In [28]:
df_top

,identifier,ticker,name,exchange,instrument,data_vendor_id,created_date,last_updated_date
1,3014145,3IINFOTECH,3I INFOTECH.,NSE,Equity,13,2020-09-26 03:42:57.891835,2020-09-26 03:42:57.891835
3,121345,3MINDIA,3M INDIA,NSE,Equity,13,2020-09-26 03:42:57.891835,2020-09-26 03:42:57.891835
4,138438660,5PAISA,,BSE,Equity,13,2020-09-26 03:42:57.891835,2020-09-26 03:42:57.891835
6,3038209,63MOONS,63 MOONS TECHNOLOGIES,NSE,Equity,13,2020-09-26 03:42:57.891835,2020-09-26 03:42:57.891835
8,1147137,AARTIDRUGS,AARTI DRUGS.,NSE,Equity,13,2020-09-26 03:42:57.891835,2020-09-26 03:42:57.891835
...,...,...,...,...,...,...,...,...
1838,1922049,ZENTEC,ZEN TECHNOLOGIES,NSE,Equity,13,2020-09-26 03:42:57.891835,2020-09-26 03:42:57.891835
1840,978177,ZODIACLOTH,ZODIAC CLOTHING CO..,NSE,Equity,13,2020-09-26 03:42:57.891835,2020-09-26 03:42:57.891835
1903,2916865,ZOTA,ZOTA HEALTH CARE,NSE,Equity,13,2020-09-26 03:42:57.891835,2020-09-26 03:42:57.891835
1842,7436801,ZUARI,ZUARI AGRO CHEMICALS,NSE,Equity,13,2020-09-26 03:42:57.891835,2020-09-26 03:42:57.891835


In [4]:
import q_credentials.db_secmaster_cred as db_secmaster_cred
import psycopg2
db_host=db_secmaster_cred.dbHost 
db_user=db_secmaster_cred.dbUser
db_password=db_secmaster_cred.dbPWD
db_name=db_secmaster_cred.dbName
conn = psycopg2.connect(host=db_host, database=db_name, user=db_user, password=db_password)

In [5]:
s3 = boto3.client('s3',endpoint_url="http://minio-image:9000",aws_access_key_id="minio-image",aws_secret_access_key="minio-image-pass")
Bucket="airflow-files"
Key="interested_tickers_india.xlsx"
read_file = s3.get_object(Bucket=Bucket, Key=Key)
df_tickers = pd.read_excel(io.BytesIO(read_file['Body'].read()),sheet_name='d')

In [12]:
if df_tickers.empty:
    print("Empty Ticker List")
else:
    # Getting the last date for each interested tickers
    sql="""select a.min_date, a.max_date, b.id as symbol_id, b.ticker from
        (select min(date_price) as min_date, max(date_price) as max_date, symbol_id
        from {}_data 
        group by symbol_id) a right join symbol b on a.symbol_id = b.id 
        where b.ticker in {} """.format('d',str(tuple(df_tickers['Tickers'])).replace(",)", ")"))
    df_ticker_last_day=pd.read_sql(sql,con=conn)

In [13]:
df_ticker_last_day

,min_date,max_date,symbol_id,ticker
0,2016-01-04,2020-10-01,1,3IINFOTECH
1,2016-01-04,2020-10-01,2,3MINDIA
2,2017-11-16,2020-10-01,3,5PAISA
3,2016-01-04,2020-10-01,4,63MOONS
4,2016-01-04,2020-10-01,5,AARTIDRUGS
...,...,...,...,...
984,2016-01-04,2020-10-01,985,ZENTEC
985,2016-01-04,2020-10-01,986,ZODIACLOTH
986,2016-01-04,2020-10-01,987,ZOTA
987,2016-01-04,2020-10-01,988,ZUARI


In [14]:
import numpy as np

In [16]:
np.array_split(df_ticker_last_day, 3)[0]

,min_date,max_date,symbol_id,ticker
0,2016-01-04,2020-10-01,1,3IINFOTECH
1,2016-01-04,2020-10-01,2,3MINDIA
2,2017-11-16,2020-10-01,3,5PAISA
3,2016-01-04,2020-10-01,4,63MOONS
4,2016-01-04,2020-10-01,5,AARTIDRUGS
...,...,...,...,...
325,2016-01-04,2020-10-01,326,GRASIM
326,2016-01-04,2020-10-01,327,GRAVITA
327,2016-01-04,2020-10-01,328,GREAVESCOT
328,2016-01-04,2020-10-01,329,GREENLAM


In [62]:
kite = zerodha_authentication.zerodha_authentication(
api_key=zerodha_cred.api_key,
api_secret=zerodha_cred.api_secret,
userid=zerodha_cred.userid,
password=zerodha_cred.password,
pin=zerodha_cred.pin
)
df_instruments=pd.DataFrame(kite.instruments())

In [63]:
df_instruments.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,540584966,2111660,EURINR20DEC77.7500CE,EURINR,0.0,2020-12-29,77.75,0.0025,1,CE,BCD-OPT,BCD
1,540578822,2111636,EURINR20DEC77.7500PE,EURINR,0.0,2020-12-29,77.75,0.0025,1,PE,BCD-OPT,BCD
2,540512774,2111378,EURINR20DEC78.0000CE,EURINR,0.0,2020-12-29,78.00,0.0025,1,CE,BCD-OPT,BCD
3,540506374,2111353,EURINR20DEC78.0000PE,EURINR,0.0,2020-12-29,78.00,0.0025,1,PE,BCD-OPT,BCD
4,540627462,2111826,EURINR20DEC78.2500CE,EURINR,0.0,2020-12-29,78.25,0.0025,1,CE,BCD-OPT,BCD


In [66]:
df_ticker_last_day=pd.merge(df_instruments,df_ticker_last_day,left_on='tradingsymbol',right_on='ticker',how='right')
df_ticker_last_day=df_ticker_last_day.sort_values(by=['tradingsymbol','exchange']).drop_duplicates(subset=['tradingsymbol'],keep='last')

In [67]:
df_ticker_last_day

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange,last_date,symbol_id,ticker
1,2714625,10604,BHARTIARTL,BHARTI AIRTEL,0.0,,0.0,0.05,1,EQ,NSE,NSE,None,195,BHARTIARTL
3,340481,1330,HDFC,HDFC,0.0,,0.0,0.05,1,EQ,NSE,NSE,None,424,HDFC
5,341249,1333,HDFCBANK,HDFC BANK,0.0,,0.0,0.05,1,EQ,NSE,NSE,None,426,HDFCBANK
7,356865,1394,HINDUNILVR,HINDUSTAN UNILEVER.,0.0,,0.0,0.05,1,EQ,NSE,NSE,None,447,HINDUNILVR
9,408065,1594,INFY,INFOSYS,0.0,,0.0,0.05,1,EQ,NSE,NSE,None,503,INFY
11,492033,1922,KOTAKBANK,KOTAK MAHINDRA BANK,0.0,,0.0,0.05,1,EQ,NSE,NSE,None,579,KOTAKBANK
13,738561,2885,RELIANCE,RELIANCE INDUSTRIES,0.0,,0.0,0.05,1,EQ,NSE,NSE,None,809,RELIANCE
15,2953217,11536,TCS,TATA CONSULTANCY SERV LT,0.0,,0.0,0.05,1,EQ,NSE,NSE,None,954,TCS


In [51]:
# str(tuple(df_tickers['Tickers'])).replace(",)", ")")

In [52]:
write_db.write_db_dataframe(df=pd.DataFrame(symbols), conn=conn, table='symbol')

NameError: name 'write_db' is not defined

In [22]:
df_data=kite.historical_data(instrument_token=128048900,from_date='2020-10-06',to_date='2020-10-07',interval='day')

In [24]:
pd.DataFrame(df_data)

""


In [62]:
portfolio=kite.holdings()

In [63]:
portfolio_df=pd.DataFrame(portfolio)
portfolio_df

,tradingsymbol,exchange,instrument_token,isin,product,price,quantity,t1_quantity,realised_quantity,authorised_quantity,opening_quantity,collateral_quantity,collateral_type,discrepancy,average_price,last_price,close_price,pnl,day_change,day_change_percentage
0,GOLDBEES,BSE,151064324,INF204KB17I5,CNC,0,600,0,600,0,600,0,,False,30.147083,43.6,43.37,8071.7502,0.23,0.53032
1,LIQUIDBEES,BSE,151064580,INF732E01037,CNC,0,325,0,325,0,325,0,,True,0.000000,1000.0,1000.00,325000.0000,0.00,0.00000


In [71]:
df_data=kite.historical_data(instrument_token=128048900,from_date='2016-11-01',to_date='2020-01-01',interval='day')

In [72]:
df_data=pd.DataFrame(df_data)

In [73]:
df_data

,date,open,high,low,close,volume
0,2016-11-01 00:00:00+05:30,41.9,42.0,40.90,41.40,9704
1,2016-11-02 00:00:00+05:30,42.0,42.0,39.65,40.00,9817
2,2016-11-03 00:00:00+05:30,40.0,40.8,38.55,38.80,6922
3,2016-11-04 00:00:00+05:30,40.7,40.7,37.30,38.00,9338
4,2016-11-07 00:00:00+05:30,38.0,39.0,38.00,38.20,4004
...,...,...,...,...,...,...
773,2019-12-26 00:00:00+05:30,13.2,13.8,13.20,13.80,1440
774,2019-12-27 00:00:00+05:30,13.5,14.1,13.25,13.42,2333
775,2019-12-30 00:00:00+05:30,13.1,13.7,13.00,13.21,2460
776,2019-12-31 00:00:00+05:30,13.5,13.5,12.90,13.06,664


In [79]:
df_data=df_data.set_index(pd.to_datetime(df_data['date']))

In [80]:
(df_data.index).date

array([datetime.date(2016, 11, 1), datetime.date(2016, 11, 2),
       datetime.date(2016, 11, 3), datetime.date(2016, 11, 4),
       datetime.date(2016, 11, 7), datetime.date(2016, 11, 8),
       datetime.date(2016, 11, 9), datetime.date(2016, 11, 10),
       datetime.date(2016, 11, 11), datetime.date(2016, 11, 15),
       datetime.date(2016, 11, 16), datetime.date(2016, 11, 17),
       datetime.date(2016, 11, 18), datetime.date(2016, 11, 21),
       datetime.date(2016, 11, 22), datetime.date(2016, 11, 23),
       datetime.date(2016, 11, 24), datetime.date(2016, 11, 25),
       datetime.date(2016, 11, 28), datetime.date(2016, 11, 29),
       datetime.date(2016, 11, 30), datetime.date(2016, 12, 1),
       datetime.date(2016, 12, 2), datetime.date(2016, 12, 5),
       datetime.date(2016, 12, 6), datetime.date(2016, 12, 7),
       datetime.date(2016, 12, 8), datetime.date(2016, 12, 9),
       datetime.date(2016, 12, 12), datetime.date(2016, 12, 13),
       datetime.date(2016, 12, 14), dat

In [80]:
import pandas as pd
import os
import io
import boto3
import pytz

In [85]:
s3 = boto3.client('s3',endpoint_url="http://minio-image:9000",aws_access_key_id="minio-image",aws_secret_access_key="minio-image-pass")
Bucket="airflow-files"
Key="interested_tickers_nse.xlsx"
read_file = s3.get_object(Bucket=Bucket, Key=Key)
df_tickers = pd.read_excel(io.BytesIO(read_file['Body'].read()),sheet_name='nse_top_1000')

In [86]:
df_tickers.head()

,Sr. No.,Symbol,Company Name,Market Capitalisation \n(In lakhs)
0,1,RELIANCE,Reliance Industries Limited,7.060358e+07
1,2,TCS,Tata Consultancy Services Limited,6.852230e+07
2,3,HINDUNILVR,Hindustan Unilever Limited,4.975841e+07
3,4,HDFCBANK,HDFC Bank Limited,4.724823e+07
4,5,HDFC,Housing Development Finance Corporation Limited,2.823571e+07


In [88]:
instrument_list=kite.instruments()
df_instruments=pd.DataFrame(instrument_list)
df_instruments.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
0,540584966,2111660,EURINR20DEC77.7500CE,EURINR,0.0,2020-12-29,77.75,0.0025,1,CE,BCD-OPT,BCD
1,540578822,2111636,EURINR20DEC77.7500PE,EURINR,0.0,2020-12-29,77.75,0.0025,1,PE,BCD-OPT,BCD
2,540512774,2111378,EURINR20DEC78.0000CE,EURINR,0.0,2020-12-29,78.00,0.0025,1,CE,BCD-OPT,BCD
3,540506374,2111353,EURINR20DEC78.0000PE,EURINR,0.0,2020-12-29,78.00,0.0025,1,PE,BCD-OPT,BCD
4,540627462,2111826,EURINR20DEC78.2500CE,EURINR,0.0,2020-12-29,78.25,0.0025,1,CE,BCD-OPT,BCD


In [90]:

df_top.head()

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange,Sr. No.,Symbol,Company Name,Market Capitalisation \n(In lakhs)
0,136352772,532628,3IINFOTECH,3I INFOTECH.,0.0,,0.0,0.01,1,EQ,BSE,BSE,903,3IINFOTECH,3i Infotech Limited,2.101638e+04
1,3014145,11774,3IINFOTECH,3I INFOTECH.,0.0,,0.0,0.05,1,EQ,NSE,NSE,903,3IINFOTECH,3i Infotech Limited,2.101638e+04
2,133989124,523395,3MINDIA,3M INDIA.,0.0,,0.0,0.05,1,EQ,BSE,BSE,97,3MINDIA,3M India Limited,2.120982e+06
3,121345,474,3MINDIA,3M INDIA,0.0,,0.0,0.05,1,EQ,NSE,NSE,97,3MINDIA,3M India Limited,2.120982e+06
4,138438660,540776,5PAISA,,0.0,,0.0,0.05,1,EQ,BSE,BSE,824,5PAISA,5Paisa Capital Limited,2.882841e+04


In [94]:
df_top=pd.merge(df_instruments,df_tickers,left_on='tradingsymbol',right_on='Symbol',how='inner')
df_top=df_top.sort_values(by=['tradingsymbol','exchange']).drop_duplicates(subset=['tradingsymbol'],keep='last')

In [95]:
df_top[df_top['exchange']=='BSE']

,instrument_token,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange,Sr. No.,Symbol,Company Name,Market Capitalisation \n(In lakhs)
4,138438660,540776,5PAISA,,0.0,,0.0,0.05,1,EQ,BSE,BSE,824,5PAISA,5Paisa Capital Limited,28828.406786
269,132459780,517421,BUTTERFLY,,0.0,,0.0,0.05,1,EQ,BSE,BSE,946,BUTTERFLY,Butterfly Gandhimathi Appliances Limited,17119.670083
424,134076420,523736,DVL,,0.0,,0.0,0.05,1,EQ,BSE,BSE,960,DVL,Dhunseri Ventures Limited,16233.973479
639,136489476,533162,HATHWAY,,0.0,,0.0,0.05,1,EQ,BSE,BSE,375,HATHWAY,Hathway Cable & Datacom Limited,238079.055250
708,128048900,500191,HMT,,0.0,,0.0,0.05,1,EQ,BSE,BSE,740,HMT,HMT Limited,39092.256790
719,136361476,532662,HTMEDIA,,0.0,,0.0,0.05,1,EQ,BSE,BSE,873,HTMEDIA,HT Media Limited,23274.831400
862,136349956,532617,JETAIRWAYS,,0.0,,0.0,0.05,1,EQ,BSE,BSE,974,JETAIRWAYS,Jet Airways (India) Limited,15506.042779
913,138537220,541161,KARDA,,0.0,,0.0,0.05,1,EQ,BSE,BSE,968,KARDA,Karda Constructions Limited,15897.750000
1040,131396868,513269,MANINDS,,0.0,,0.0,0.05,1,EQ,BSE,BSE,872,MANINDS,Man Industries (India) Limited,23412.252550
1057,139065092,543223,MAXINDIA,MAX INDIA,0.0,,0.0,0.05,1,EQ,BSE,BSE,435,MAXINDIA,Max India Limited,163885.984280


In [96]:
df_top.columns

Index(['instrument_token', 'exchange_token', 'tradingsymbol', 'name',
       'last_price', 'expiry', 'strike', 'tick_size', 'lot_size',
       'instrument_type', 'segment', 'exchange', 'Sr. No.', 'Symbol',
       'Company Name', 'Market Capitalisation \n(In lakhs)'],
      dtype='object')

In [ ]:
'ticker':symbol['name'],'exchange':'Forex','instrument':'Forex','name':symbol['displayName'],'data_vendor_id':data_vendor_id,'created_date':now,'last_updated_date':now